In [2]:
import pandas as pd
import os
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm


In [3]:
class PlantDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.classes = sorted(self.data['class'].unique())
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['image_path']
        label = self.class_to_idx[self.data.iloc[idx]['class']]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [6]:
train_csv = "train_data.csv"
train_dataset = PlantDataset(train_csv, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

val_csv = "test_data.csv"
val_dataset = PlantDataset(val_csv, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

print("Number of classes:", len(train_dataset.classes))


Number of classes: 38


In [16]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 38)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


m:\GitHub\plnat-disease-classifiacation\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
m:\GitHub\plnat-disease-classifiacation\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
epochs = 10

model.train()
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    model.train()
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
    
    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total
    
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


Epoch 1/10 [Val]: 100%|██████████| 138/138 [04:09<00:00,  1.81s/it]


Epoch 1/10 | Train Loss: 0.1556, Train Acc: 95.41% | Val Loss: 0.0944, Val Acc: 96.93%


Epoch 2/10 [Val]: 100%|██████████| 138/138 [04:05<00:00,  1.78s/it]


Epoch 2/10 | Train Loss: 0.0526, Train Acc: 98.38% | Val Loss: 0.0676, Val Acc: 97.83%


Epoch 3/10 [Train]:   9%|▉         | 50/550 [01:49<18:18,  2.20s/it]


KeyboardInterrupt: 

In [18]:
import torch
from tqdm import tqdm

def evaluate_accuracy(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    acc = 100 * correct / total
    return acc

train_acc = evaluate_accuracy(model, train_loader, device)
print(f"Train Accuracy: {train_acc:.2f}%")

test_acc = evaluate_accuracy(model, val_loader, device)
print(f"Test Accuracy: {test_acc:.2f}%")

torch.save(model.state_dict(), "resnet18_plant_classifier.pth")
print("Model saved as resnet18_plant_classifier.pth")


Evaluating: 100%|██████████| 550/550 [16:16<00:00,  1.78s/it]


Train Accuracy: 98.04%


Evaluating: 100%|██████████| 138/138 [04:06<00:00,  1.79s/it]

Test Accuracy: 97.39%
Model saved as resnet18_plant_classifier.pth


In [7]:
import json

with open('classes.json', 'w') as f:
    json.dump(train_dataset.classes, f)